In [1]:
pip install pyspark==3.2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyspark as ps

In [3]:
file="C:/Users/nvgkv/Downloads/BBC News Train.csv"

In [6]:
#sc.stop()
conf = ps.SparkConf().setAll([('spark.executor.memory', '16g'), ('spark.driver.memory', '16g')])
sc = ps.SparkContext(conf=conf)

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
from pyspark.sql import SQLContext

sql_context = SQLContext(sc)
voxDf = sql_context.read.format('com.databricks.spark.csv').option('header','true').option("delimiter", ",").load(file)

In [ ]:
prepprocesDF=voxDf.drop("title","author","published_date","updated_on","slug","blurb","ArticleId")

In [ ]:
voxDf.show(5)
prepprocesDF.show(5)

In [ ]:
from pyspark.sql.functions import regexp_replace

prepprocesDF = prepprocesDF.withColumn("text", regexp_replace("Text", "<[^>]+>", ""))



In [ ]:
prepprocesDF.show(10)

In [ ]:
prepprocesDF=prepprocesDF.drop("body")
prepprocesDF=prepprocesDF.dropna()

In [ ]:
trainDF,testDF=prepprocesDF.randomSplit([0.75, 0.25], seed=2000)


In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StringIndexer,CountVectorizer,RegexTokenizer

In [ ]:
from pyspark.ml.feature import NGram, StopWordsRemover

In [ ]:
from pyspark.ml import Pipeline, PipelineModel
#tokenizer = Tokenizer(inputCol="text", outputCol="words")
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
# Define NGram transformer

stopwordremover=StopWordsRemover(inputCol="words", outputCol="wordsWithStopwordsfree")
#ngram = NGram(n=2, inputCol="wordsWithStopwordsfree", outputCol="bigrams")
# bag of words count
countVectors = CountVectorizer(inputCol="wordsWithStopwordsfree", outputCol="cf", vocabSize=20000, minDF=5)
#hashtf = HashingTF(numFeatures=2 ** 16, inputCol="wordsWithStopwordsfree", outputCol="cf")
idf = IDF(inputCol="cf", outputCol="features", minDocFreq=5)
label_string_idx = StringIndexer(inputCol="Category", outputCol="label").setHandleInvalid("skip")
pipeline = Pipeline(stages=[regexTokenizer,stopwordremover,countVectors,idf,label_string_idx])


In [ ]:
prepprocesDF.show(5)

In [ ]:
pipeline_fit = pipeline.fit(prepprocesDF)

In [ ]:
pipeline_fit = pipeline.fit(prepprocesDF)
#pipeline_fit.save("pipleline_model_path")
train_df = pipeline_fit.transform(trainDF)
test_df = pipeline_fit.transform(testDF)
#prepprocesDF = pipeline_fit.transform(prepprocesDF)

In [ ]:
test_df.show(50)

In [ ]:
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel,NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#lr =  LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lr=NaiveBayes(smoothing=1)
lr_model = lr.fit(train_df)

 

In [ ]:
predictions = lr_model.transform(test_df)

In [ ]:
predictions.show(500)

In [ ]:


from pyspark.ml.evaluation import MulticlassClassificationEvaluator


evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

In [ ]:
#tweeter data pipe line
regexTokenizertw = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
# Define NGram transformer

stopwordremovertw=StopWordsRemover(inputCol="words", outputCol="wordsWithStopwordsfree")
#ngram = NGram(n=2, inputCol="wordsWithStopwordsfree", outputCol="bigrams")
# bag of words count
countVectorstw = CountVectorizer(inputCol="wordsWithStopwordsfree", outputCol="cf", vocabSize=20000, minDF=5)
#hashtf = HashingTF(numFeatures=2 ** 16, inputCol="wordsWithStopwordsfree", outputCol="tf")
idftw = IDF(inputCol="cf", outputCol="features", minDocFreq=5)
#label_string_idx = StringIndexer(inputCol="Category", outputCol="label").setHandleInvalid("skip")
pipelinetw = Pipeline(stages=[regexTokenizertw,stopwordremovertw,countVectorstw,idftw])

In [ ]:
tweets1 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load('/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/CategoryPredictModelandData/tuesday.csv')
tweets2 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load('/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/CategoryPredictModelandData/tuesday1.csv')

In [ ]:
import functools
  
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

In [ ]:
tweets = unionAll([tweets1, tweets2])
print(tweets.count())


In [ ]:
tweets = tweets.where(tweets._c0!='null')

In [ ]:
print(tweets.count())

In [ ]:
tweets.show()

In [ ]:
tweets=tweets.selectExpr("_c0 as text")

In [ ]:
pipeline_fittw = pipelinetw.fit(prepprocesDF)

In [ ]:
test_df = pipeline_fittw.transform(testDF)

In [ ]:
test_df.show(600,truncate=False)

In [ ]:
tweets=pipeline_fittw.transform(tweets)

In [ ]:
tweets.show(6)

In [ ]:
tweetprediction = lr_model.transform(tweets)

In [ ]:
tweetprediction.show(10)

In [ ]:
tweetprediction=tweetprediction.drop("words","wordsWithStopwordsfree","cf","features","rawPrediction","probability")

In [ ]:
tweetprediction.show(n=60,truncate=False,vertical=False)

In [ ]:
from pyspark.sql.functions import col,when
tweetprediction=tweetprediction.withColumn("prediction", when(col("prediction") == 0.0, "sport").when(col("prediction") ==1.0, "business").when(col("prediction") ==2.0, "political").when(col("prediction") ==3.0, "entertainment").otherwise("tech"))

In [ ]:
tweetprediction.show(n=60,truncate=False)

In [ ]:
tweetprediction.write.csv("/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/tuesdaycategoryPredictedDataset")